<a href="https://colab.research.google.com/github/StokedDude/CANSLIM-signal-booster/blob/main/CANSLIM_Booster_V6_CIO_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🚀 CANSLIM Signal Booster V6 + Personal CIO System
**Complete Trading Intelligence Platform**

---

## Features:

### CANSLIM Signal Booster V6
- ✅ Multi-source ingestion (IBD, DeepVue, Primus, Finviz, X, watchlist)
- ✅ Convergence scoring (multi-source confirmation)
- ✅ Dynamic trust tiers (A/B/C/D auto-classification)
- ✅ Context flags (extended/proper setup/volume/timing)
- ✅ Instant ticker lookup
- ✅ Non-IBD ticker handling (technical-only scoring)

### Personal CIO System
- 📊 Daily Brief generator (pre-market intelligence)
- 📈 Portfolio health tracker
- 🎯 Market regime detector
- 📉 Trade vs hold advisor
- 📋 Source performance analytics

---

**Last Updated:** February 2026  
**Version:** 6.0

## 📦 1. Setup & Installation

In [1]:
# Install required packages
!pip install -q yfinance pandas numpy scipy scikit-learn pyqlib tabulate

import pandas as pd
import numpy as np
import yfinance as yf
from datetime import datetime, timedelta
import warnings
from typing import Dict, List, Tuple, Optional
from tabulate import tabulate
import json
from collections import defaultdict
from sklearn.ensemble import RandomForestClassifier
from google.colab import files
import io

warnings.filterwarnings('ignore')

print("✅ Setup complete!")
print(f"📅 Current date: {datetime.now().strftime('%Y-%m-%d')}")

✅ Setup complete!
📅 Current date: 2026-02-03


## 📂 2. Data Ingestion - Upload IBD Exports

**Upload your IBD CSV exports:**
- IBD 50
- IBD 250
- Sector Leaders
- Big Cap 20
- IPO Leaders
- Any other MarketSurge exports

**Expected columns:** Ticker, Company, Composite, RS Rating, EPS Rating, SMR, Acc/Dist, etc.

In [2]:
# Global data storage
ibd_universe = pd.DataFrame()
all_sources_data = defaultdict(list)
ticker_lookup_db = {}

# Upload and merge IBD exports
print("📤 Upload your IBD CSV files (you can upload multiple files)")
print("Click 'Choose Files' and select all your IBD exports...\n")

uploaded = files.upload()

ibd_dataframes = []

for filename, content in uploaded.items():
    try:
        df = pd.read_csv(io.BytesIO(content))

        # Standardize column names (handle variations)
        df.columns = df.columns.str.strip().str.replace(' ', '_')

        # Try to identify ticker column
        ticker_col = None
        for col in ['Symbol', 'Ticker', 'Stock', 'SYMBOL', 'TICKER']:
            if col in df.columns:
                ticker_col = col
                break

        if ticker_col:
            df = df.rename(columns={ticker_col: 'Ticker'})
            df['Source_File'] = filename
            ibd_dataframes.append(df)
            print(f"✅ Loaded {filename}: {len(df)} stocks")
        else:
            print(f"⚠️  Skipped {filename}: Could not find ticker column")

    except Exception as e:
        print(f"❌ Error loading {filename}: {e}")

# Merge all IBD data
if ibd_dataframes:
    ibd_universe = pd.concat(ibd_dataframes, ignore_index=True)

    # Remove duplicates (keep first occurrence)
    ibd_universe = ibd_universe.drop_duplicates(subset='Ticker', keep='first')

    # Clean ticker symbols
    ibd_universe['Ticker'] = ibd_universe['Ticker'].str.strip().str.upper()

    print(f"\n━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━")
    print(f"✅ IBD UNIVERSE LOADED")
    print(f"━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━")
    print(f"Total unique stocks: {len(ibd_universe)}")
    print(f"Files processed: {len(ibd_dataframes)}")
    print(f"\nSample stocks: {', '.join(ibd_universe['Ticker'].head(10).tolist())}")

    # Show available columns
    print(f"\nAvailable data fields: {', '.join(ibd_universe.columns.tolist())}")
else:
    print("❌ No IBD data loaded. Please upload CSV files.")

📤 Upload your IBD CSV files (you can upload multiple files)
Click 'Choose Files' and select all your IBD exports...



Saving IBD Big Cap 20.csv to IBD Big Cap 20 (2).csv
Saving MarketSurge Growth 250.csv to MarketSurge Growth 250 (2).csv
Saving Today's Industry Performance _ NEW HIGHS.csv to Today's Industry Performance _ NEW HIGHS (2).csv
Saving Daily % Change.csv to Daily % Change (2).csv
Saving IBD 50 Index.csv to IBD 50 Index (2).csv
Saving 197 Industry Groups.csv to 197 Industry Groups (2).csv
✅ Loaded IBD Big Cap 20 (2).csv: 20 stocks
✅ Loaded MarketSurge Growth 250 (2).csv: 299 stocks
✅ Loaded Today's Industry Performance _ NEW HIGHS (2).csv: 57 stocks
✅ Loaded Daily % Change (2).csv: 197 stocks
✅ Loaded IBD 50 Index (2).csv: 50 stocks
✅ Loaded 197 Industry Groups (2).csv: 197 stocks

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
✅ IBD UNIVERSE LOADED
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
Total unique stocks: 547
Files processed: 6

Sample stocks: FIX, APH, RL, LLY, HEI, KGC, RGLD, NVT, FN, WPM

Available data fields: Order, Ticker, Name, Comp_Rating, EPS_Rating, RS_Rating, Ind_Group_RS, SMR_Rating, A/D_Rating, 52-W

## 🔧 3. Technical Factor Calculations (Qlib-style)

Calculate technical factors for all stocks in the universe

In [4]:
class TechnicalFactorEngine:
    """Calculate CANSLIM technical factors"""

    def __init__(self):
        self.market_data = {}

    def get_price_data(self, ticker: str, period: str = "6mo") -> pd.DataFrame:
        """Fetch price data with caching"""
        if ticker not in self.market_data:
            try:
                data = yf.download(ticker, period=period, progress=False)
                if not data.empty:
                    self.market_data[ticker] = data
            except:
                return pd.DataFrame()
        return self.market_data.get(ticker, pd.DataFrame())

    def calculate_rs(self, ticker: str, weeks: int = 13) -> float:
        """Calculate Relative Strength vs SPY"""
        try:
            stock_data = self.get_price_data(ticker)
            spy_data = self.get_price_data("SPY")

            if stock_data.empty or spy_data.empty:
                return 50  # Neutral

            days = weeks * 5  # Trading days

            stock_return = (stock_data['Close'].iloc[-1] / stock_data['Close'].iloc[-days] - 1) * 100
            spy_return = (spy_data['Close'].iloc[-1] / spy_data['Close'].iloc[-days] - 1) * 100

            # Normalize to 0-99 scale (simplified)
            relative_perf = stock_return - spy_return
            rs_score = min(99, max(1, 50 + (relative_perf * 2)))

            return round(rs_score)
        except:
            return 50

    def calculate_volume_trend(self, ticker: str) -> str:
        """Analyze volume pattern (Expanding/Contracting/Neutral)"""
        try:
            data = self.get_price_data(ticker)
            if data.empty or 'Volume' not in data.columns:
                return "Unknown"

            # Compare recent 5 days vs previous 20 days
            recent_vol = data['Volume'].iloc[-5:].mean()
            avg_vol = data['Volume'].iloc[-25:-5].mean()

            if recent_vol > avg_vol * 1.2:
                return "Expanding"
            elif recent_vol < avg_vol * 0.8:
                return "Contracting"
            else:
                return "Neutral"
        except:
            return "Unknown"

    def calculate_pivot_distance(self, ticker: str) -> Tuple[float, str]:
        """Calculate distance from 52-week high (pivot proxy)"""
        try:
            data = self.get_price_data(ticker, period="1y")
            if data.empty:
                return 0, "Unknown"

            current_price = data['Close'].iloc[-1]
            high_52w = data['High'].max()

            distance_pct = ((current_price / high_52w) - 1) * 100

            if distance_pct >= -5:
                status = "PROPER SETUP"
            elif distance_pct >= -10:
                status = "NEAR PIVOT"
            elif distance_pct >= -15:
                status = "PULLBACK"
            else:
                status = "EXTENDED DOWN"

            return round(distance_pct, 1), status
        except:
            return 0, "Unknown"

    def calculate_atr(self, ticker: str, period: int = 14) -> float:
        """Calculate Average True Range for volatility/stop placement"""
        try:
            data = self.get_price_data(ticker)
            if data.empty:
                return 0

            high_low = data['High'] - data['Low']
            high_close = abs(data['High'] - data['Close'].shift())
            low_close = abs(data['Low'] - data['Close'].shift())

            true_range = pd.concat([high_low, high_close, low_close], axis=1).max(axis=1)
            atr = true_range.rolling(window=period).mean().iloc[-1]

            return round(atr, 2)
        except:
            return 0

    def determine_timeframe(self, atr: float, pivot_distance: float) -> str:
        """Determine if stock is suitable for daytrade/swing/position"""
        if atr > 5 and abs(pivot_distance) < 5:
            return "DAYTRADE"
        elif abs(pivot_distance) < 5:
            return "SWING"
        elif abs(pivot_distance) < 10:
            return "POSITION"
        else:
            return "WATCH"

# Initialize engine
tech_engine = TechnicalFactorEngine()

print("✅ Technical Factor Engine initialized")
print("Ready to calculate RS, volume patterns, pivot distance, ATR, etc.")

✅ Technical Factor Engine initialized
Ready to calculate RS, volume patterns, pivot distance, ATR, etc.


## 🎯 4. Process IBD Universe - Add Technical Overlays

In [5]:
from __future__ import annotations

from concurrent.futures import ThreadPoolExecutor, as_completed
from typing import Any, Dict, List, Optional, Tuple

import pandas as pd


def overlay_technical_factors(
    ibd_universe: pd.DataFrame,
    tech_engine: Any,
    chunk_size: int = 50,
    max_workers: int = 12,
) -> pd.DataFrame:
    """
    Add technical factor columns to an IBD universe DataFrame.

    Improvements vs original:
    - Faster row iteration (itertuples)
    - Parallel per-ticker computation (ThreadPoolExecutor)
    - Per-ticker error isolation (one failure won't break the run)
    - Adds Status/Error columns so you can audit failures
    """

    if ibd_universe is None or ibd_universe.empty:
        print("❌ No IBD universe to process. Please upload IBD exports first.")
        return ibd_universe

    if "Ticker" not in ibd_universe.columns:
        raise ValueError("ibd_universe must contain a 'Ticker' column")

    print("🔄 Processing IBD universe with technical factors...")
    print(f"Analyzing {len(ibd_universe)} stocks\n")

    # Pre-fetch benchmark once (warm caches, speed up RS calculations)
    # Assumes tech_engine caches internally; if not, this is still harmless.
    _ = tech_engine.get_price_data("SPY")

    total_stocks = len(ibd_universe)
    tickers = ibd_universe["Ticker"].astype(str).str.upper().tolist()

    def compute_one(ticker: str) -> Dict[str, Any]:
        """
        Compute factors for one ticker.
        This is isolated so exceptions are captured per ticker.
        """
        try:
            rs_calc = tech_engine.calculate_rs(ticker)
            volume_trend = tech_engine.calculate_volume_trend(ticker)
            pivot_dist, pivot_status = tech_engine.calculate_pivot_distance(ticker)
            atr = tech_engine.calculate_atr(ticker)
            timeframe = tech_engine.determine_timeframe(atr, pivot_dist)

            return {
                "Ticker": ticker,
                "RS_Calculated": rs_calc,
                "Volume_Trend": volume_trend,
                "Pivot_Distance_%": pivot_dist,
                "Pivot_Status": pivot_status,
                "ATR": atr,
                "Timeframe": timeframe,
                "Status": "OK",
                "Error": "",
            }
        except Exception as e:
            # Keep row so merges are stable; you can later filter Status != OK
            return {
                "Ticker": ticker,
                "RS_Calculated": None,
                "Volume_Trend": None,
                "Pivot_Distance_%": None,
                "Pivot_Status": None,
                "ATR": None,
                "Timeframe": None,
                "Status": "FAIL",
                "Error": f"{type(e).__name__}: {e}",
            }

    results: List[Dict[str, Any]] = []

    # Process in chunks to provide progress feedback without spamming
    for start in range(0, total_stocks, chunk_size):
        chunk_tickers = tickers[start : start + chunk_size]

        # Threaded execution is best when tech_engine does network/disk I/O per ticker.
        # If your tech_engine is pure CPU, use ProcessPoolExecutor instead.
        with ThreadPoolExecutor(max_workers=max_workers) as ex:
            futures = {ex.submit(compute_one, t): t for t in chunk_tickers}
            for fut in as_completed(futures):
                results.append(fut.result())

        processed = min(start + chunk_size, total_stocks)
        pct = (processed / total_stocks) * 100
        print(f"Progress: {processed}/{total_stocks} ({pct:.1f}%)")

    technical_df = pd.DataFrame(results)

    # Merge technical overlay back into IBD universe
    out = ibd_universe.copy()
    out["Ticker"] = out["Ticker"].astype(str).str.upper()
    out = out.merge(technical_df, on="Ticker", how="left")

    print("\n✅ Technical overlay complete!")
    print(f"Total stocks processed: {len(out)}")
    print(f"Failures: {(out['Status'] == 'FAIL').sum() if 'Status' in out.columns else 0}")

    return out


## 🏆 5. Tier Classification System

In [6]:
def classify_tier(row: pd.Series) -> str:
    """Assign A/B/C/D tier based on CANSLIM criteria"""

    # Get RS rating (prefer IBD's if available, otherwise use calculated)
    rs_ibd_cols = ['RS_Rating', 'RS', 'Relative_Strength']
    rs = None
    for col in rs_ibd_cols:
        if col in row.index and pd.notna(row[col]):
            rs = row[col]
            break
    if rs is None:
        rs = row.get('RS_Calculated', 50)

    # Get Composite rating
    comp_cols = ['Composite', 'Composite_Rating', 'Comp']
    composite = None
    for col in comp_cols:
        if col in row.index and pd.notna(row[col]):
            composite = row[col]
            break
    if composite is None:
        composite = 50

    pivot_status = row.get('Pivot_Status', 'Unknown')

    # A-TIER: IBD 50 quality + proper setup
    if rs >= 90 and composite >= 90 and pivot_status == "PROPER SETUP":
        return "A-TIER"

    # B-TIER: IBD 250 quality OR strong technical
    elif rs >= 80 and composite >= 80:
        return "B-TIER"

    # C-TIER: Moderate quality
    elif rs >= 70 or composite >= 70:
        return "C-TIER"

    # D-TIER: Low quality
    else:
        return "D-TIER"

if not ibd_universe.empty:
    # Apply tier classification
    ibd_universe['Tier'] = ibd_universe.apply(classify_tier, axis=1)

    # Add IBD verification flag
    ibd_universe['Verification'] = "IBD-VERIFIED"

    # Summary stats
    tier_counts = ibd_universe['Tier'].value_counts()

    print("━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━")
    print("📊 TIER CLASSIFICATION")
    print("━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━")
    for tier in ['A-TIER', 'B-TIER', 'C-TIER', 'D-TIER']:
        count = tier_counts.get(tier, 0)
        print(f"{tier}: {count} stocks")

    print("\n✅ Tier classification complete!")

    # Build lookup database
    for idx, row in ibd_universe.iterrows():
        ticker_lookup_db[row['Ticker']] = row.to_dict()

    print(f"✅ Lookup database built with {len(ticker_lookup_db)} stocks")
else:
    print("❌ No data to classify")

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
📊 TIER CLASSIFICATION
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
A-TIER: 0 stocks
B-TIER: 0 stocks
C-TIER: 346 stocks
D-TIER: 201 stocks

✅ Tier classification complete!
✅ Lookup database built with 547 stocks


## 📥 6. Multi-Source Input - Other Idea Sources

Enter tickers from other sources (DeepVue, Primus, Finviz, X mentions, etc.)

In [7]:
def add_source_tickers(source_name: str, tickers_input: str):
    """Add tickers from a source"""
    # Parse ticker input (comma or space separated)
    tickers = [t.strip().upper() for t in tickers_input.replace(',', ' ').split() if t.strip()]

    for ticker in tickers:
        all_sources_data[ticker].append(source_name)

    return tickers

print("═══════════════════════════════════════════════")
print("📥 ADD TICKERS FROM OTHER SOURCES")
print("═══════════════════════════════════════════════")
print("\nEnter tickers from each source (comma or space separated)")
print("Leave blank and press Enter to skip\n")

# DeepVue
deepvue_input = input("🔹 DeepVue True Market Leaders: ")
if deepvue_input:
    deepvue_tickers = add_source_tickers("DeepVue", deepvue_input)
    print(f"   Added {len(deepvue_tickers)} tickers from DeepVue")

# Primus
primus_input = input("🔹 Primus Pre-Market Scan: ")
if primus_input:
    primus_tickers = add_source_tickers("Primus", primus_input)
    print(f"   Added {len(primus_tickers)} tickers from Primus")

# Finviz
finviz_input = input("🔹 Finviz Scan Results: ")
if finviz_input:
    finviz_tickers = add_source_tickers("Finviz", finviz_input)
    print(f"   Added {len(finviz_tickers)} tickers from Finviz")

# X/Twitter mentions
x_input = input("🔹 X/Twitter Mentions: ")
if x_input:
    x_tickers = add_source_tickers("X_Mentions", x_input)
    print(f"   Added {len(x_tickers)} tickers from X")

# Personal watchlist
watchlist_input = input("🔹 Personal Watchlist: ")
if watchlist_input:
    watchlist_tickers = add_source_tickers("Watchlist", watchlist_input)
    print(f"   Added {len(watchlist_tickers)} tickers from Watchlist")

# Summary
total_unique = len(all_sources_data)
print(f"\n✅ Total unique tickers from all sources: {total_unique}")

═══════════════════════════════════════════════
📥 ADD TICKERS FROM OTHER SOURCES
═══════════════════════════════════════════════

Enter tickers from each source (comma or space separated)
Leave blank and press Enter to skip

🔹 DeepVue True Market Leaders: GE
   Added 1 tickers from DeepVue
🔹 Primus Pre-Market Scan: 
🔹 Finviz Scan Results: TER SNDK PLTR DVA LITE
   Added 5 tickers from Finviz
🔹 X/Twitter Mentions: 
🔹 Personal Watchlist: 

✅ Total unique tickers from all sources: 6


## ⭐ 7. Convergence Scoring Engine

In [8]:
# Source weights (customize as needed)
SOURCE_WEIGHTS = {
    'IBD_50': 3,
    'IBD_250': 2,
    'DeepVue': 2,
    'Primus': 2,
    'Finviz': 1,
    'X_Mentions': 1,
    'Watchlist': 1
}

def calculate_convergence_score(ticker: str, sources: List[str]) -> Tuple[int, str]:
    """Calculate weighted convergence score"""
    total_weight = sum(SOURCE_WEIGHTS.get(source, 1) for source in sources)
    max_possible = sum(SOURCE_WEIGHTS.values())

    # Normalize to 0-10 scale
    score = min(10, int((total_weight / max_possible) * 10))

    # Stars for display
    if score >= 8:
        stars = "⭐⭐⭐"
    elif score >= 6:
        stars = "⭐⭐"
    elif score >= 4:
        stars = "⭐"
    else:
        stars = ""

    return score, stars

# Add IBD tickers to sources
if not ibd_universe.empty:
    for ticker in ibd_universe['Ticker'].tolist():
        # Determine which IBD list (simplified)
        row = ibd_universe[ibd_universe['Ticker'] == ticker].iloc[0]

        # Check RS to determine IBD 50 vs 250
        rs_cols = ['RS_Rating', 'RS', 'Relative_Strength']
        rs = None
        for col in rs_cols:
            if col in row.index and pd.notna(row[col]):
                rs = row[col]
                break

        if rs and rs >= 90:
            all_sources_data[ticker].append('IBD_50')
        else:
            all_sources_data[ticker].append('IBD_250')

# Calculate convergence for all tickers
convergence_data = []

for ticker, sources in all_sources_data.items():
    score, stars = calculate_convergence_score(ticker, sources)

    convergence_data.append({
        'Ticker': ticker,
        'Sources': ', '.join(sources),
        'Source_Count': len(sources),
        'Convergence_Score': score,
        'Stars': stars
    })

convergence_df = pd.DataFrame(convergence_data).sort_values('Convergence_Score', ascending=False)

print("━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━")
print("⭐ CONVERGENCE SCORING")
print("━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━")
print(f"Total tickers analyzed: {len(convergence_df)}")
print(f"High conviction (⭐⭐⭐): {len(convergence_df[convergence_df['Convergence_Score'] >= 8])}")
print(f"Medium conviction (⭐⭐): {len(convergence_df[convergence_df['Convergence_Score'].between(6, 7)])}")
print(f"Low conviction (⭐): {len(convergence_df[convergence_df['Convergence_Score'] < 6])}")

print("\n✅ Convergence analysis complete!")

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
⭐ CONVERGENCE SCORING
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
Total tickers analyzed: 550
High conviction (⭐⭐⭐): 0
Medium conviction (⭐⭐): 0
Low conviction (⭐): 550

✅ Convergence analysis complete!


## 🔍 8. Instant Ticker Lookup Function

## 📊 9. Daily Brief Generator - Personal CIO System

In [10]:
from __future__ import annotations

from typing import Any, Dict

import pandas as pd


def lookup_ticker(ticker: str):
    """
    Instant ticker quality lookup.

    Assumes these exist in the outer scope:
      - ticker_lookup_db: dict[str, dict]
      - convergence_df: pd.DataFrame
      - tech_engine: object with calculate_rs / calculate_volume_trend / calculate_pivot_distance / calculate_atr
      - pd imported as pandas
    """
    ticker = str(ticker).strip().upper()

    print("\n" + "=" * 50)
    print(f"📊 TICKER LOOKUP: {ticker}")
    print("=" * 50)

    # Helper: safe convergence lookup (avoids KeyError / iloc crashes)
    def _conv_row(t: str) -> pd.DataFrame:
        """Return convergence rows for ticker, or empty df if not available."""
        if convergence_df is None or convergence_df.empty or "Ticker" not in convergence_df.columns:
            return pd.DataFrame()
        # Normalize case for matching
        return convergence_df[convergence_df["Ticker"].astype(str).str.upper() == t]

    def _safe_first(df: pd.DataFrame, col: str, default: Any = "") -> Any:
        """Return first cell in df[col] if possible; otherwise default."""
        if df is None or df.empty or col not in df.columns:
            return default
        val = df[col].iloc[0]
        return default if pd.isna(val) else val

    # Check if in IBD database
    if ticker in ticker_lookup_db:
        data: Dict[str, Any] = ticker_lookup_db[ticker]

        # Convergence info
        conv_info = _conv_row(ticker)
        sources = _safe_first(conv_info, "Sources", default="IBD only")
        stars = _safe_first(conv_info, "Stars", default="")

        tier = data.get("Tier", "UNKNOWN")
        print(f"\n✅ {tier} - IBD VERIFIED")
        print(f"\nConviction: {stars}")
        print(f"Sources: {sources}")

        # Ratings
        print("\n📈 CANSLIM Ratings:")
        for col in ["Composite", "RS_Rating", "EPS_Rating", "SMR", "Acc/Dist"]:
            if col in data and pd.notna(data[col]):
                print(f"  {col}: {data[col]}")

        # Technical
        print("\n📊 Technical Analysis:")
        print(f"  Pivot Status: {data.get('Pivot_Status', 'N/A')}")
        print(f"  Distance from Pivot: {data.get('Pivot_Distance_%', 'N/A')}%")
        print(f"  Volume Trend: {data.get('Volume_Trend', 'N/A')}")
        print(f"  ATR: ${data.get('ATR', 'N/A')}")
        print(f"  Timeframe: {data.get('Timeframe', 'N/A')}")

        # Recommendation
        pivot_status = str(data.get("Pivot_Status", "")).upper()

        print("\n💡 Recommendation:")
        if tier == "A-TIER" and pivot_status == "PROPER SETUP":
            print("  🟢 HIGH CONVICTION - Ready to trade")
        elif tier == "A-TIER":
            print("  🟡 QUALITY STOCK - Wait for better entry")
        elif tier == "B-TIER" and pivot_status == "PROPER SETUP":
            print("  🟡 WATCHABLE - Consider smaller position")
        elif tier == "B-TIER":
            print("  🟡 WATCH - Set alerts for entry")
        else:
            print("  🔴 PASS - Low conviction")

    else:
        print("\n⚠️  NOT IN IBD DATABASE")

        # Check if in other sources
        conv_info = _conv_row(ticker)

        if not conv_info.empty:
            sources = _safe_first(conv_info, "Sources", default="Other sources")
            print(f"\nFound in: {sources}")
            print("\nCalculating technical scores...")

            # Calculate technical-only scores (wrap to avoid one ticker killing the function)
            try:
                rs = tech_engine.calculate_rs(ticker)
                volume = tech_engine.calculate_volume_trend(ticker)
                pivot_dist, pivot_status = tech_engine.calculate_pivot_distance(ticker)
                atr = tech_engine.calculate_atr(ticker)
            except Exception as e:
                print(f"\n❌ Technical calc failed: {type(e).__name__}: {e}")
                print("\n" + "=" * 50)
                return

            print("\n📊 TECHNICAL-ONLY Analysis:")
            print(f"  RS (estimated): {rs}")
            print(f"  Volume: {volume}")
            print(f"  Pivot Status: {pivot_status} ({pivot_dist}%)")
            print(f"  ATR: ${atr}")

            # Grade
            pivot_status_u = str(pivot_status).upper()
            if rs >= 80 and pivot_status_u == "PROPER SETUP":
                grade = "B-TIER (Technical-only)"
                rec = "🟡 WATCHABLE - No fundamental verification, smaller size"  # <-- fixed extra ')'
            elif rs >= 70:
                grade = "C-TIER (Technical-only)"
                rec = "🟠 DAYTRADE ONLY - Tight stops, no fundamentals"
            else:
                grade = "D-TIER"
                rec = "🔴 PASS - Weak on all metrics"

            print(f"\nGrade: {grade}")
            print(f"\n💡 Recommendation: {rec}")
        else:
            print("\n❌ Not found in any source")
            print("Consider adding to watchlist or ignoring")

    print("\n" + "=" * 50)


# Example usage (keep this out of libraries/modules if you plan to import it elsewhere)
print("\n🔍 INSTANT TICKER LOOKUP")
print("Enter any ticker to get instant quality report")
print("Example: lookup_ticker('NVDA')\n")

test_ticker = input("Enter ticker to lookup (or press Enter to skip): ").strip()
if test_ticker:
    lookup_ticker(test_ticker)



🔍 INSTANT TICKER LOOKUP
Enter any ticker to get instant quality report
Example: lookup_ticker('NVDA')

Enter ticker to lookup (or press Enter to skip): NVDA

📊 TICKER LOOKUP: NVDA

✅ C-TIER - IBD VERIFIED

Conviction: 
Sources: IBD_250

📈 CANSLIM Ratings:
  RS_Rating: 84.0
  EPS_Rating: 99.0

📊 Technical Analysis:
  Pivot Status: N/A
  Distance from Pivot: N/A%
  Volume Trend: N/A
  ATR: $N/A
  Timeframe: N/A

💡 Recommendation:
  🔴 PASS - Low conviction



In [12]:
from __future__ import annotations

from datetime import datetime
from typing import Any, Dict, Optional

import pandas as pd


def generate_daily_brief(
    top_n_daytrades: int = 5,
    top_n_swings: int = 3,
    *,
    tech_engine: Any,
    ibd_universe: Optional[pd.DataFrame] = None,
    convergence_df: Optional[pd.DataFrame] = None,
    all_sources_data: Optional[Dict[str, Any]] = None,
    ticker_lookup_db: Optional[Dict[str, Any]] = None,
) -> None:
    """
    Generate comprehensive pre-market daily brief.

    Fixes vs your version:
    - Imports datetime
    - No bare except; prints the exception type/message
    - Handles missing/empty dataframes
    - Safe merges (only if required columns exist)
    - Avoids KeyError on missing columns
    - Handles missing globals by injecting dependencies
    """

    current_date = datetime.now().strftime("%A, %B %d, %Y")

    # Normalize optional inputs
    ibd_universe = ibd_universe if isinstance(ibd_universe, pd.DataFrame) else pd.DataFrame()
    convergence_df = convergence_df if isinstance(convergence_df, pd.DataFrame) else pd.DataFrame()
    all_sources_data = all_sources_data or {}
    ticker_lookup_db = ticker_lookup_db or {}

    print("\n" + "=" * 60)
    print("📋 DAILY TRADING BRIEF")
    print(f"{current_date} | Pre-Market")
    print("=" * 60)

    # ---------------------------
    # Market Regime (simple proxy)
    # ---------------------------
    try:
        spy_data = tech_engine.get_price_data("SPY", period="1mo")
        if spy_data is None or len(spy_data) < 6 or "Close" not in spy_data.columns:
            raise ValueError("SPY data missing or insufficient (need >=6 rows with 'Close')")

        spy_return_5d = ((spy_data["Close"].iloc[-1] / spy_data["Close"].iloc[-5]) - 1) * 100

        vix_data = tech_engine.get_price_data("^VIX", period="1mo")
        if vix_data is not None and not vix_data.empty and "Close" in vix_data.columns:
            vix_current = float(vix_data["Close"].iloc[-1])
        else:
            vix_current = 15.0  # fallback default

        if spy_return_5d > 1 and vix_current < 15:
            regime = "Trending Up ✅"
            sizing = "Full position sizing OK"
        elif spy_return_5d < -1 or vix_current > 20:
            regime = "Choppy / Distribution ⚠️"
            sizing = "Reduce size 50%, selective trades only"
        else:
            regime = "Neutral"
            sizing = "Normal position sizing"

        print(f"\nMARKET REGIME: {regime}")
        print(f"Recommendation: {sizing}")
        print(f"SPY (5-day): {spy_return_5d:+.1f}%")
        print(f"VIX: {vix_current:.1f}")

    except Exception as e:
        print("\nMARKET REGIME: Unable to fetch (check connection)")
        print(f"Reason: {type(e).__name__}: {e}")

    print("\n" + "━" * 60)

    # ---------------------------
    # Helpers for safe operations
    # ---------------------------
    def has_cols(df: pd.DataFrame, cols: list[str]) -> bool:
        """Return True if df contains all required columns."""
        return isinstance(df, pd.DataFrame) and all(c in df.columns for c in cols)

    def safe_merge_convergence(df: pd.DataFrame) -> pd.DataFrame:
        """
        Merge convergence fields if available.
        Avoids KeyErrors when columns are missing.
        """
        needed = ["Ticker", "Stars", "Sources", "Convergence_Score"]
        if df.empty:
            return df

        if not has_cols(df, ["Ticker"]):
            return df

        if has_cols(convergence_df, needed):
            return df.merge(convergence_df[needed], on="Ticker", how="left")

        # If convergence_df exists but missing some columns, merge only what exists
        if not convergence_df.empty and "Ticker" in convergence_df.columns:
            available = ["Ticker"] + [c for c in ["Stars", "Sources", "Convergence_Score"] if c in convergence_df.columns]
            if len(available) > 1:
                return df.merge(convergence_df[available], on="Ticker", how="left")

        return df

    def safe_sort(df: pd.DataFrame) -> pd.DataFrame:
        """Sort by convergence then tier if possible; otherwise return df unchanged."""
        if df.empty:
            return df

        sort_cols = []
        ascending = []

        if "Convergence_Score" in df.columns:
            sort_cols.append("Convergence_Score")
            ascending.append(False)

        if "Tier" in df.columns:
            sort_cols.append("Tier")
            # Your original used ascending True for Tier; keep it
            ascending.append(True)

        if sort_cols:
            return df.sort_values(sort_cols, ascending=ascending)

        return df

    # ---------------------------
    # Top Daytrades
    # ---------------------------
    print(f"\n🎯 TOP {top_n_daytrades} DAYTRADE SETUPS\n")

    if not ibd_universe.empty and has_cols(ibd_universe, ["Timeframe", "Tier", "Ticker"]):
        daytrade_candidates = ibd_universe[
            (ibd_universe["Timeframe"] == "DAYTRADE")
            & (ibd_universe["Tier"].isin(["A-TIER", "B-TIER"]))
        ].copy()

        daytrade_candidates = safe_merge_convergence(daytrade_candidates)
        daytrade_candidates = safe_sort(daytrade_candidates).head(top_n_daytrades)

        if daytrade_candidates.empty:
            print("   No daytrade candidates matching filters\n")
        else:
            for i, row in enumerate(daytrade_candidates.itertuples(index=False), 1):
                rowd = row._asdict()  # convert namedtuple to dict for .get usage

                rs = rowd.get("RS_Rating", rowd.get("RS_Calculated", "N/A"))
                comp = rowd.get("Composite", "N/A")

                print(f"{i}. {rowd.get('Ticker')} - {rowd.get('Tier')} {rowd.get('Stars', '')}")
                print(f"   Sources: {rowd.get('Sources', 'IBD only')}")
                print(f"   RS: {rs} | Composite: {comp}")
                print(f"   Setup: {rowd.get('Pivot_Status', 'N/A')} ({rowd.get('Pivot_Distance_%', 'N/A')}%)")
                print(f"   Volume: {rowd.get('Volume_Trend', 'N/A')} | ATR: ${rowd.get('ATR', 'N/A')}")
                print()
    else:
        print("   No data available (missing ibd_universe or required columns)\n")

    print("━" * 60)

    # ---------------------------
    # Top Swings
    # ---------------------------
    print(f"\n📈 TOP {top_n_swings} SWING SETUPS\n")

    if not ibd_universe.empty and has_cols(ibd_universe, ["Timeframe", "Tier", "Ticker"]):
        swing_candidates = ibd_universe[
            (ibd_universe["Timeframe"].isin(["SWING", "POSITION"]))
            & (ibd_universe["Tier"].isin(["A-TIER", "B-TIER"]))
        ].copy()

        swing_candidates = safe_merge_convergence(swing_candidates)
        swing_candidates = safe_sort(swing_candidates).head(top_n_swings)

        if swing_candidates.empty:
            print("   No swing candidates matching filters\n")
        else:
            for i, row in enumerate(swing_candidates.itertuples(index=False), 1):
                rowd = row._asdict()
                rs = rowd.get("RS_Rating", rowd.get("RS_Calculated", "N/A"))
                comp = rowd.get("Composite", "N/A")

                print(f"{i}. {rowd.get('Ticker')} - {rowd.get('Tier')} {rowd.get('Stars', '')}")
                print(f"   Sources: {rowd.get('Sources', 'IBD only')}")
                print(f"   RS: {rs} | Composite: {comp}")
                print(f"   Setup: {rowd.get('Pivot_Status', 'N/A')}")
                print()
    else:
        print("   No data available (missing ibd_universe or required columns)\n")

    print("━" * 60)

    # ---------------------------
    # Watch List
    # ---------------------------
    print("\n👁️  WATCH LIST (Set alerts, don't trade yet)\n")

    if not ibd_universe.empty and has_cols(ibd_universe, ["Timeframe", "Tier", "Ticker"]):
        watch_candidates = ibd_universe[
            (ibd_universe["Timeframe"] == "WATCH")
            & (ibd_universe["Tier"].isin(["A-TIER", "B-TIER"]))
        ].copy()

        if not watch_candidates.empty:
            watch_list = watch_candidates["Ticker"].head(10).astype(str).tolist()
            print(f"   {len(watch_candidates)} stocks in proper bases waiting for catalyst:")
            print(f"   {', '.join(watch_list)}")
            if len(watch_candidates) > 10:
                print(f"   ...and {len(watch_candidates) - 10} more")
        else:
            print("   No stocks currently on watch")
    else:
        print("   No data available (missing ibd_universe or required columns)")
    print()

    print("━" * 60)

    # ---------------------------
    # Filtered Out
    # ---------------------------
    print("\n🚫 FILTERED OUT TODAY\n")

    total_tickers = len(all_sources_data) if isinstance(all_sources_data, dict) else 0
    in_ibd = (
        len([t for t in all_sources_data.keys() if t in ticker_lookup_db])
        if isinstance(all_sources_data, dict) and isinstance(ticker_lookup_db, dict)
        else 0
    )
    filtered = total_tickers - in_ibd

    if total_tickers == 0:
        print("   No source universe loaded (all_sources_data is empty/missing)")
    elif filtered > 0:
        print(f"   {filtered} tickers from various sources")
        print(f"   Reasons: Not in IBD ({filtered}), Extended, or Weak setup")
        print("   Action: IGNORED")
    else:
        print("   All tickers passed quality filter")

    print("\n" + "=" * 60)
    print("✅ Daily Brief Complete")
    print("=" * 60)


# Example call (requires you to pass the dependencies explicitly)
# print("\n🔄 Generating Daily Brief...\n")
# generate_daily_brief(
#     tech_engine=tech_engine,
#     ibd_universe=ibd_universe,
#     convergence_df=convergence_df,
#     all_sources_data=all_sources_data,
#     ticker_lookup_db=ticker_lookup_db,
# )


## 📊 10. Export & Save Functions

In [13]:
def export_daily_brief_csv():
    """Export daily brief to CSV for external use"""

    if ibd_universe.empty:
        print("❌ No data to export")
        return

    # Merge with convergence
    export_df = ibd_universe.merge(
        convergence_df[['Ticker', 'Stars', 'Sources', 'Convergence_Score']],
        on='Ticker',
        how='left'
    )

    # Sort by tier and convergence
    export_df = export_df.sort_values(['Tier', 'Convergence_Score'], ascending=[True, False])

    # Select key columns
    key_cols = ['Ticker', 'Tier', 'Stars', 'Sources', 'Convergence_Score',
                'Timeframe', 'Pivot_Status', 'Pivot_Distance_%', 'Volume_Trend', 'ATR']

    # Add any IBD rating columns that exist
    for col in ['Composite', 'RS_Rating', 'EPS_Rating', 'SMR', 'Acc/Dist']:
        if col in export_df.columns:
            key_cols.append(col)

    export_subset = export_df[[col for col in key_cols if col in export_df.columns]]

    # Save
    filename = f"daily_brief_{datetime.now().strftime('%Y%m%d')}.csv"
    export_subset.to_csv(filename, index=False)

    print(f"✅ Daily brief exported to {filename}")
    print(f"   Total stocks: {len(export_subset)}")
    print(f"   A-TIER: {len(export_subset[export_subset['Tier'] == 'A-TIER'])}")
    print(f"   B-TIER: {len(export_subset[export_subset['Tier'] == 'B-TIER'])}")

    # Download file
    files.download(filename)

# Export option
export_choice = input("\nExport daily brief to CSV? (y/n): ")
if export_choice.lower() == 'y':
    export_daily_brief_csv()


Export daily brief to CSV? (y/n): y
✅ Daily brief exported to daily_brief_20260203.csv
   Total stocks: 547
   A-TIER: 0
   B-TIER: 0


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## 🔄 11. Quick Commands - Interactive Interface

In [ ]:
print("\n" + "="*60)
print("🎮 INTERACTIVE COMMAND CENTER")
print("="*60)
print("\nAvailable Commands:")
print("  1. lookup_ticker('SYMBOL') - Quick ticker analysis")
print("  2. generate_daily_brief() - Regenerate full brief")
print("  3. View A-TIER stocks only")
print("  4. View multi-source convergence (⭐⭐⭐)")
print("  5. Export to CSV")
print("\nExamples:")
print("  lookup_ticker('NVDA')")
print("  generate_daily_brief()")
print("="*60)

# Quick views
def show_a_tier():
    """Show all A-TIER stocks"""
    if not ibd_universe.empty:
        a_tier = ibd_universe[ibd_universe['Tier'] == 'A-TIER'].copy()

        if not a_tier.empty:
            # Merge with convergence
            a_tier = a_tier.merge(
                convergence_df[['Ticker', 'Stars', 'Convergence_Score']],
                on='Ticker',
                how='left'
            )

            a_tier = a_tier.sort_values('Convergence_Score', ascending=False)

            print("\n━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━")
            print(f"⭐ A-TIER STOCKS ({len(a_tier)} total)")
            print("━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━\n")

            for idx, row in a_tier.iterrows():
                print(f"{row['Ticker']} {row.get('Stars', '')} - {row.get('Pivot_Status', 'N/A')} | {row.get('Timeframe', 'N/A')}")
        else:
            print("\nNo A-TIER stocks found")

def show_high_conviction():
    """Show high conviction multi-source stocks"""
    high_conv = convergence_df[convergence_df['Convergence_Score'] >= 8]

    print("\n━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━")
    print(f"⭐⭐⭐ HIGH CONVICTION ({len(high_conv)} stocks)")
    print("━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━\n")

    for idx, row in high_conv.iterrows():
        ticker = row['Ticker']
        tier = ticker_lookup_db.get(ticker, {}).get('Tier', 'Unknown')
        print(f"{ticker} - {tier} | Sources: {row['Sources']}")

# Interactive menu
while True:
    choice = input("\nEnter command (or 'q' to quit): ").strip()

    if choice.lower() == 'q':
        print("👋 Goodbye!")
        break
    elif choice == '3':
        show_a_tier()
    elif choice == '4':
        show_high_conviction()
    elif choice == '5':
        export_daily_brief_csv()
    elif choice.startswith('lookup_ticker'):
        # Extract ticker from command
        try:
            ticker = choice.split("'")[1]
            lookup_ticker(ticker)
        except:
            print("❌ Invalid format. Use: lookup_ticker('SYMBOL')")
    elif choice == 'generate_daily_brief()' or choice == '2':
        generate_daily_brief()
    else:
        print("❌ Unknown command. Try: 2, 3, 4, 5, or lookup_ticker('SYMBOL')")


🎮 INTERACTIVE COMMAND CENTER

Available Commands:
  1. lookup_ticker('SYMBOL') - Quick ticker analysis
  2. generate_daily_brief() - Regenerate full brief
  3. View A-TIER stocks only
  4. View multi-source convergence (⭐⭐⭐)
  5. Export to CSV

Examples:
  lookup_ticker('NVDA')
  generate_daily_brief()


## 📝 12. Summary & Next Steps

In [15]:
print("\n" + "="*60)
print("📋 SYSTEM SUMMARY")
print("="*60)

print("\n✅ Systems Active:")
print("   • CANSLIM Signal Booster V6")
print("   • Personal CIO System")
print("   • Multi-source convergence scoring")
print("   • Instant ticker lookup")

print("\n📊 Data Loaded:")
print(f"   • IBD Universe: {len(ibd_universe)} stocks")
print(f"   • A-TIER: {len(ibd_universe[ibd_universe['Tier'] == 'A-TIER']) if not ibd_universe.empty else 0}")
print(f"   • Multi-source tickers: {len(all_sources_data)}")
print(f"   • High conviction (⭐⭐⭐): {len(convergence_df[convergence_df['Convergence_Score'] >= 8])}")

print("\n🎯 Quick Access:")
print("   • lookup_ticker('SYMBOL') - Instant analysis")
print("   • generate_daily_brief() - Full pre-market brief")
print("   • show_a_tier() - View A-TIER stocks")
print("   • show_high_conviction() - Multi-source plays")

print("\n📈 Workflow:")
print("   1. Upload IBD exports daily (5-10 min)")
print("   2. Add tickers from other sources (2 min)")
print("   3. Generate daily brief (instant)")
print("   4. Focus on top 5 daytrades + top 3 swings")
print("   5. Quick lookups during market hours")

print("\n💡 Pro Tips:")
print("   • Trust A-TIER + ⭐⭐⭐ convergence most")
print("   • Use lookup_ticker() for unknown tickers from Primus")
print("   • Export CSV for external tracking/analysis")
print("   • Re-run brief if new data comes in")

print("\n" + "="*60)
print("🚀 System Ready - Happy Trading!")
print("="*60)


📋 SYSTEM SUMMARY

✅ Systems Active:
   • CANSLIM Signal Booster V6
   • Personal CIO System
   • Multi-source convergence scoring
   • Instant ticker lookup

📊 Data Loaded:
   • IBD Universe: 547 stocks
   • A-TIER: 0
   • Multi-source tickers: 550
   • High conviction (⭐⭐⭐): 0

🎯 Quick Access:
   • lookup_ticker('SYMBOL') - Instant analysis
   • generate_daily_brief() - Full pre-market brief
   • show_a_tier() - View A-TIER stocks
   • show_high_conviction() - Multi-source plays

📈 Workflow:
   1. Upload IBD exports daily (5-10 min)
   2. Add tickers from other sources (2 min)
   3. Generate daily brief (instant)
   4. Focus on top 5 daytrades + top 3 swings
   5. Quick lookups during market hours

💡 Pro Tips:
   • Trust A-TIER + ⭐⭐⭐ convergence most
   • Use lookup_ticker() for unknown tickers from Primus
   • Export CSV for external tracking/analysis
   • Re-run brief if new data comes in

🚀 System Ready - Happy Trading!
